In [2]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import mne

In [3]:
def getOverlap(a,b):
    """
    get the number of ms overlapped between 2 time windows
    """
    return max(0,min(a[1],b[1]) - max(a[0],b[0]))

In [4]:
# trial
suj = '23'
day = '1'

In [5]:
eeg_dir             = '../../EEG/'
annotation_dir      = 'annotations/'

f_name = f'suj{suj}_l2nap_day{day}.vhdr'

raw = mne.io.read_raw_brainvision(eeg_dir + f_name, preload=True)

# set the EOG channels
channel_types = {'LOc':'eog','ROc':'eog','Aux1':'misc'}
raw.set_channel_types(channel_types)

raw_ref, _  = mne.set_eeg_reference(raw,
                                    ref_channels = 'average',
                                    projection   = True,)
raw_ref.apply_proj() # it might tell you it already has been re-referenced, but do it anyway

# read standard montage - montage is important for visualization
montage = mne.channels.make_standard_montage('standard_1020');#montage.plot();#montage.plot()
raw.set_montage(montage)

Extracting parameters from ../../EEG/suj23_l2nap_day1.vhdr...
Setting channel info structure...
Reading 0 ... 1804149  =      0.000 ...  1804.149 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


<RawBrainVision | suj23_l2nap_day1.eeg, 64 x 1804150 (1804.1 s), ~881.0 MB, data loaded>

In [6]:
# some hyper-hyper-parameters
time_steps  = 250  # in miliseconds
window_size = 1000 # in miliseconds

In [13]:
# create time segments for cutting overlapping windows
df_events           = pd.read_csv(os.path.join(annotation_dir,
                                               f'suj{suj}_day{day}_annotations.txt'))

# we cut off the last part of EEG when no particular events
spindle_events      = df_events[df_events['Annotation'] == 'spindle']
kcomplex_events     = df_events[df_events['Annotation'] == 'k-complex']
stage_2_sleep_events= df_events[df_events['Annotation'] == 'Marker:Markoff: 2']

# we only look at the data from when the first 2nd stage sleep started
if len(stage_2_sleep_events) > 1:
    print('stage 2 sleep annotations are provided')
    tmin            = np.min(stage_2_sleep_events['Onset'].values)
else:
    tmin            = 0
# and we stop looking at the data when the last spindle, kcomplex, or 2nd stage stops,
# whichever one happens the latest
tmax                = np.max([spindle_events['Onset'].values.max(),
                              kcomplex_events['Onset'].values.max() + 1,
                              stage_2_sleep_events['Onset'].values.max() + 1,
                            ]) * raw.info['sfreq']
onsets              = np.arange(start = tmin,
                                stop  = tmax,
                                step  = time_steps)

offsets             = onsets + window_size

windows             = np.vstack([onsets,offsets]).T.astype(int)

# ***label spindles***
# if a segement of EEG contains a spindle time stamp, it is labeled "1"
# so we directly use the Pandas DataFrame in the name of "spindle_events" we created above
spindle_time_stamps = np.array(spindle_events['Onset'].values * 1000,
                               dtype = 'int')
labels              = []
# let's define all spindle lasted for 1.5 seconds and the annotated time stamp was put on the .25 second location
intervals = [[item-250,item+1250] for item in spindle_time_stamps]

# if the segmented window overlap any spindle window, it is defined as a spindle segment
# but, we want to define the "overlap" better, so I also add a term "tolerate"
# only if the overlapping is more than some minimum requirement -- tolerate -- we can say it is a spindle
tol = 20 # in milliseconds
for window in tqdm(windows):
    if np.sum([getOverlap(window,item) for item in intervals]) > tol:
        labels.append(1)
    else:
        labels.append(0)

event_id            = {'spindle':1,'no spindle':0}
events              = np.vstack([onsets,
                                 np.zeros(onsets.shape),
                                 np.array(labels)]).T.astype(int)


stage 2 sleep annotations are provided


100%|██████████| 7082/7082 [00:00<00:00, 41807.09it/s]


In [17]:
event_id['spindle']

1